In [7]:
# -*- coding: utf-8 -*-
"""
Created on Mon Dec 11 02:33:01 2017
@author: rmisra
"""

import json
import gzip
import numpy as np
import string
import random
import operator
from collections import defaultdict
from sklearn.linear_model import LogisticRegression
from pylmnn.lmnn import LargeMarginNearestNeighbor as LMNN
from plots import plot_comparison
from collections import defaultdict

def parseData(file):
    for l in open(file,'r'):
        yield json.loads(l)
        
def remove_punctuation(text):
    return ''.join([c.lower() for c in text if c not in set(string.punctuation)])

np.random.seed(5)

## Implementation of 1-LV

### Method Proposed in https://dl.acm.org/citation.cfm?id=3109891

### Read and split data

In [8]:
data = list(parseData('datasets/renttherunway_final_data.json'))

In [9]:
data[10]

{'fit': 'small',
 'user_id': '185966',
 'bust size': '34b',
 'item_id': '1077123',
 'weight': '135lbs',
 'rating': '8',
 'rented for': 'party',
 'review_text': "The dress arrived with a small hole in the beading on the front but wasn't too noticeable. Glad I was able to get two sizes because the 4 was a little tight and would've made for an uncomfortable night of dancing! ",
 'body type': 'athletic',
 'review_summary': "It was fun to wear a dress I wouldn't normally buy! ",
 'category': 'dress',
 'height': '5\' 3"',
 'size': 12,
 'age': '33',
 'review_date': 'January 2, 2018'}

In [10]:
random.seed(1)
random.shuffle(data)

train_data = data[:int(0.8*len(data))]
val_data = data[int(0.8*len(data)):int(0.9*len(data))]
test_data = data[int(0.9*len(data)):]

len(train_data), len(val_data), len(test_data)

(154035, 19254, 19255)

### Index user and items

In [12]:
item_data = {}
item_index = {}
user_index = {}
user_data = {}
u_index = 0
i_index = 0
for r in train_data:
    if r['item_id'] + '|' + str(r['size']) not in item_data:
        item_data[r['item_id'] + '|' + str(r['size'])] = [r]
        item_index[r['item_id'] + '|' + str(r['size'])] = i_index
        i_index += 1
    else:
        item_data[r['item_id'] + '|' + str(r['size'])].append(r)
        
    if r['user_id'] not in user_data:
        user_data[r['user_id']] = [r]
        user_index[r['user_id']] = u_index
        u_index += 1
    else:
        user_data[r['user_id']].append(r)

In [13]:
len(user_data), len(user_index), len(item_data), len(item_index)

(90510, 90510, 28938, 28938)

### Training

#### Initialize items' true sizes with respective catalog sizes and users' true sizes randomly

In [19]:
true_size_item = np.zeros(len(item_data))
true_size_cust = np.zeros(len(user_data))
w = 1; b_1 = -1; b_2 = 1; lamda = 2

for item in item_data:
    true_size_item[item_index[item]] = int(item.split('|')[1])

In [ ]:
from sklearn.linear_model import LogisticRegression
def calc_auc():
    train_features = []
    train_labels = []
    for r in train_data:
        fe = []
        fe.append(true_size_cust[user_index[r['user_id']]])
        fe.append(true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]])
        train_features.append(fe)

        if 'small' in r['fit']:
            train_labels.append(0)
        elif 'fit' in r['fit']:
            train_labels.append(1)
        elif 'large' in r['fit']:
            train_labels.append(2)

    c = 1
    clf_1LV = LogisticRegression(fit_intercept=True, multi_class='ovr', C=c)
    clf_1LV.fit(train_features, train_labels)

    test_features = []; test_labels = []; test_labels_auc = []
    for r in test_data:
        fe = []
        try:
            fe.append(true_size_cust[user_index[r['user_id']]])
        except KeyError:
            fe.append(np.mean(true_size_cust))
        try:
            fe.append(true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]])
        except KeyError:
            fe.append(np.mean(true_size_item))

        test_features.append(fe)
        label = [0, 0, 0]
        if 'small' in r['fit']:
            test_labels.append(0)
            label[0] = 1
        elif 'fit' in r['fit']:
            test_labels.append(1)
            label[1] = 1
        elif 'large' in r['fit']:
            test_labels.append(2)
            label[2] = 1
        test_labels_auc.append(label)

    test_labels_auc = np.array(test_labels_auc)

    from sklearn.metrics import roc_auc_score
    from sklearn.preprocessing import label_binarize

    pred = clf_1LV.predict_proba(test_features)
    AUC = []
    for i in range(3):
        AUC.append(roc_auc_score(test_labels_auc[:,i], pred[:,i], average='weighted'))
    print('Average AUC', np.mean(AUC), AUC)
    
def f(s,t):
    return w*(s-t)

def cal_loss_user(user, cust_size):
    loss = 0
    for r in user_data[user]:
        if 'small' in r['fit']:
            loss += max(0, 1 - f(cust_size, true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]]) + b_2)
        elif 'fit' in r['fit']:
            loss += max(0, 1 + f(cust_size, true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]]) - b_2)
            loss += max(0, 1 - f(cust_size, true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]]) + b_1)
        elif 'large' in r['fit']:
            loss += max(0, 1 + f(cust_size, true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]]) - b_1)
    return loss
            
def cal_loss_item(item, product_size):
    loss = 0
    for r in item_data[item]:
        if 'small' in r['fit']:
            loss += max(0, 1 - f(true_size_cust[user_index[r['user_id']]], product_size) + b_2)
        elif 'fit' in r['fit']:
            loss += max(0, 1 + f(true_size_cust[user_index[r['user_id']]], product_size) - b_2)
            loss += max(0, 1 - f(true_size_cust[user_index[r['user_id']]], product_size) + b_1)
        elif 'large' in r['fit']:
            loss += max(0, 1 + f(true_size_cust[user_index[r['user_id']]], product_size) - b_1)
    return loss

def total_loss():
    loss = 0
    for item in item_data:
        for r in item_data[item]:
            product_size = true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]]
            if 'small' in r['fit']:
                loss += max(0, 1 - f(true_size_cust[user_index[r['user_id']]], product_size) + b_2)
            elif 'fit' in r['fit']:
                loss += max(0, 1 + f(true_size_cust[user_index[r['user_id']]], product_size) - b_2)
                loss += max(0, 1 - f(true_size_cust[user_index[r['user_id']]], product_size) + b_1)
            elif 'large' in r['fit']:
                loss += max(0, 1 + f(true_size_cust[user_index[r['user_id']]], product_size) - b_1)
    return loss

for iterr in range(0,220):
    
    ## Phase 1
    for user in user_data:
        candidate_sizes = []
        for r in user_data[user]:
            if 'small' in r['fit']:
                candidate_sizes.append(true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]] + ((b_2+1)/w))
            elif 'fit' in r['fit']:
                candidate_sizes.append(true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]] + ((b_1+1)/w))
                candidate_sizes.append(true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]] + ((b_2-1)/w))
            elif 'large' in r['fit']:
                candidate_sizes.append(true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]] + ((b_1-1)/w))

        flag = 0
        candidate_sizes = list(set(candidate_sizes))
        candidate_sizes = sorted(candidate_sizes)

        if len(candidate_sizes) == 1:
            true_size_cust[user_index[user]] = candidate_sizes[0]
        else:
            for s in range(1, len(candidate_sizes)):
                slope = (cal_loss_user(user, candidate_sizes[s]) - cal_loss_user(user, candidate_sizes[s-1]))/(candidate_sizes[s] - candidate_sizes[s-1])
                if slope>=0:
                    flag=1
                    true_size_cust[user_index[user]] = candidate_sizes[s-1]
                    break

            if flag==0:
                true_size_cust[user_index[user]] = candidate_sizes[-1]

    ## Phase 2
    for item in item_data:
        candidate_sizes = []
        for r in item_data[item]:
            if 'small' in r['fit']:
                candidate_sizes.append(true_size_cust[user_index[r['user_id']]] - ((b_2+1)/w))
            elif 'fit' in r['fit']:
                candidate_sizes.append(true_size_cust[user_index[r['user_id']]] - ((b_1+1)/w))
                candidate_sizes.append(true_size_cust[user_index[r['user_id']]] - ((b_2-1)/w))
            elif 'large' in r['fit']:
                candidate_sizes.append(true_size_cust[user_index[r['user_id']]] - ((b_1-1)/w))

        flag = 0
        candidate_sizes = list(set(candidate_sizes))
        candidate_sizes = sorted(candidate_sizes)
        if len(candidate_sizes) == 1:
            true_size_item[item_index[item]] = candidate_sizes[0]
        else:
            for s in range(1, len(candidate_sizes)):
                slope = (cal_loss_item(item, candidate_sizes[s]) - cal_loss_item(item, candidate_sizes[s-1]))/(candidate_sizes[s] - candidate_sizes[s-1])
                if slope>=0:
                    flag=1
                    true_size_item[item_index[item]] = candidate_sizes[s-1]
                    break

            if flag==0:
                true_size_item[item_index[item]] = candidate_sizes[-1]

    ## Phase 3
    learning_rate = 0.0000005/np.sqrt(iterr+1)
    grad_w = 0
    grad_b1 = 0
    grad_b2 = 0
    for r in train_data:
        s = true_size_cust[user_index[r['user_id']]]
        t = true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]]

        if 'small' in r['fit']:
            A = 1 - f(s, t) + b_2
            if A>0:
                grad_w += -1*(s - t)
                grad_b2 += 1
        elif 'fit' in r['fit']:
            B = 1 + f(s, t) - b_2
            C = 1 - f(s, t) + b_1
            if B>0:
                grad_w += (s - t)
                grad_b2 += -1
            if C>0:
                grad_w += -1*(s - t)
                grad_b1 += 1
        elif 'large' in r['fit']:
            D = 1 + f(s, t) - b_1
            if D>0:
                grad_w += (s - t)
                grad_b1 += -1

    w -= learning_rate*(grad_w + 2*lamda*w)
    b_1 -= learning_rate*(grad_b1 + 2*lamda*b_1)
    b_2 -= learning_rate*(grad_b2 + 2*lamda*b_2)
    if iterr%5 == 0:
        print(iterr, total_loss())
        calc_auc()

### Use the above learned features in a classifier

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize

train_features = []; train_labels = []
for r in train_data:
    fe = []
    fe.append(true_size_cust[user_index[r['user_id']]])
    fe.append(true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]])
    train_features.append(fe)

    if 'small' in r['fit']:
        train_labels.append(0)
    elif 'fit' in r['fit']:
        train_labels.append(1)
    elif 'large' in r['fit']:
        train_labels.append(2)

c = 1
clf_1LV = LogisticRegression(fit_intercept=True, multi_class='ovr', C=c)
clf_1LV.fit(train_features, train_labels)

test_features = []; test_labels = []; test_labels_auc = []
for r in val_data:
    fe = []
    try:
        u = user_index[r['user_id']]
        fe.append(true_size_cust[u])
    except KeyError:
        fe.append(np.mean(true_size_cust))
    try:
        fe.append(true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]])
    except KeyError:
        fe.append(np.mean(true_size_item))

    test_features.append(fe)
    label = [0, 0, 0]
    if 'small' in r['fit']:
        test_labels.append(0)
        label[0] = 1
    elif 'fit' in r['fit']:
        test_labels.append(1)
        label[1] = 1
    elif 'large' in r['fit']:
        test_labels.append(2)
        label[2] = 1
    test_labels_auc.append(label)

test_labels_auc = np.array(test_labels_auc)

pred = clf_1LV.predict_proba(test_features)
AUC = []
for i in range(3):
    AUC.append(roc_auc_score(test_labels_auc[:,i], pred[:,i], average='weighted'))
print('Average AUC', np.mean(AUC), AUC)

## Implementation of K-LF-ML

### Create users' and items' (parent and child) indices

In [ ]:
item_data = {}; user_data = {}
item_index = {}; user_index = {}; p_item_index = {} ## p_ is for parent 
u_index = 0; i_index = 0; p_i_index = 0
sizes = {}
s_index = 0

for r in data:
    if r['size'] not in sizes:
        sizes[r['size']] = s_index
        s_index += 1

    if r['item_id'] + '|' + str(r['size']) not in item_data:
        item_data[r['item_id'] + '|' + str(r['size'])] = [r]
        item_index[r['item_id'] + '|' + str(r['size'])] = i_index
        i_index += 1
    else:
        item_data[r['item_id'] + '|' + str(r['size'])].append(r)
        
    if r['item_id'] not in p_item_index:
        p_item_index[r['item_id']] = p_i_index
        p_i_index += 1
        
    if r['user_id'] not in user_data:
        user_data[r['user_id']] = [r]
        user_index[r['user_id']] = u_index
        u_index += 1
    else:
        user_data[r['user_id']].append(r)
len(user_data), len(user_index), len(item_data), len(item_index), len(p_item_index)

### For monotonicity constraints, for each product, record smaller and larger sized products, if any

In [ ]:
product_sizes = {}; product_sizes_rev = {}

for r in data:
    if r['item_id'] not in product_sizes:
        product_sizes[r['item_id']] = [r['size']]
    else:
        product_sizes[r['item_id']].append(r['size'])
        
for k in product_sizes:
    product_sizes[k] = list(sorted(set(product_sizes[k])))
    product_sizes_rev[k] = list(sorted(set(product_sizes[k]), reverse=True))
    
product_smaller = {}
product_larger = {}

for k in product_sizes:
    for i in range(len(product_sizes[k])):
        if len(product_sizes[k]) == 1:
            product_smaller[item_index[k + '|' + str(product_sizes[k][i])]] = -1
            product_larger[item_index[k + '|' + str(product_sizes[k][i])]] = -1
        elif i == 0:
            product_smaller[item_index[k + '|' + str(product_sizes[k][i])]] = -1
            product_larger[item_index[k + '|' + str(product_sizes[k][i])]] = item_index[k + '|' + str(product_sizes[k][i+1])]
        elif i == len(product_sizes[k]) - 1:
            product_smaller[item_index[k + '|' + str(product_sizes[k][i])]] = item_index[k + '|' + str(product_sizes[k][i-1])]
            product_larger[item_index[k + '|' + str(product_sizes[k][i])]] = -1
        else:
            product_smaller[item_index[k + '|' + str(product_sizes[k][i])]] = item_index[k + '|' + str(product_sizes[k][i-1])]
            product_larger[item_index[k + '|' + str(product_sizes[k][i])]] = item_index[k + '|' + str(product_sizes[k][i+1])]

### Decompose fit semantics using latent factor model

In [ ]:
def f(a,b_i,b_u,s,t):
    return np.dot(w, np.concatenate(([a, b_u, b_i], np.multiply(s, t))))

K = 10; learning_rate = 0.000005; alpha = 1; 
true_size_item = np.random.normal(size = (len(item_index), K))*0.1
true_size_cust = np.random.normal(size = (len(user_index), K), loc=1, scale=0.1)
bias_i = np.random.normal(size = (len(item_index)))*0.1
bias_u = np.random.normal(size = (len(user_index)))*0.1
b_1 = -5; b_2 = 5; lamda = 2; w = np.ones(K+3)

for k in product_sizes:
    start = len(product_sizes[k])
    for size in product_sizes[k]:
        true_size_item[item_index[k + '|' + str(size)]] = np.random.normal(size=(1,K), loc=start, scale=0.1)
        start -= 1

In [ ]:
def calc_auc():
    train_features = []; train_labels = []
    for r in train_data:
        fe = []
        fe.append(w[1]*bias_u[user_index[r['user_id']]])
        fe.append(w[2]*bias_i[p_item_index[r['item_id']]])
        fe.extend(np.multiply(w[3:], np.multiply(true_size_cust[user_index[r['user_id']]], true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]])))

        train_features.append(fe)

        if 'small' in r['fit']:
            train_labels.append(1)
        elif 'fit' in r['fit']:
            train_labels.append(2)
        elif 'large' in r['fit']:
            train_labels.append(3)

    clf = LogisticRegression(fit_intercept=True, multi_class='ovr')
    clf.fit(train_features, train_labels)

    test_features = []; test_labels = []; test_labels_auc = []
    for r in test_data:
        fe = []
        flag = 0
        fe.append(w[1]*bias_u[user_index[r['user_id']]])
        fe.append(w[2]*bias_i[p_item_index[r['item_id']]])
        fe.extend(np.multiply(w[3:], np.multiply(true_size_cust[user_index[r['user_id']]], true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]])))

        test_features.append(fe)
        label = [0, 0, 0]
        if 'small' in r['fit']:
            test_labels.append(1)
            label[0] = 1
        elif 'fit' in r['fit']:
            test_labels.append(2)
            label[1] = 1
        elif 'large' in r['fit']:
            test_labels.append(3)
            label[2] = 1
        test_labels_auc.append(label)

    test_labels_auc = np.array(test_labels_auc)

    from sklearn.metrics import roc_auc_score
    pred = clf.predict_proba(test_features)
    AUC = []
    for i in range(3):
        AUC.append(roc_auc_score(test_labels_auc[:,i], pred[:,i], average='weighted'))
    print('Average AUC', np.mean(AUC), AUC)

In [ ]:
def calc_metric_auc():
    U = true_size_cust; V = true_size_item
    def prepare_features(data):
        X = []
        Y = []
        Y_auc = []
        item_l = []
        item_n = []
        items = {}
        item_count = defaultdict(int)
        item_small = defaultdict(int)
        frac_small = []
        for r in data:
            fe = []
            fe.append(w[1]*bias_u[user_index[r['user_id']]])
            fe.append(w[2]*bias_i[p_item_index[r['item_id']]])
            fe.extend(np.multiply(w[3:], np.multiply(true_size_cust[user_index[r['user_id']]], true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]])))
            X.append(fe)
            item_l.append(np.multiply(w[3:],V[item_index[r['item_id'] + '|' + str(r['size'])]]))
            item_n.append(str(r['category']))
            items[r['item_id'] + '|' + str(r['size'])] = 1
            if 'small' in r['fit']:
                Y.append(0)
                Y_auc.append([1,0,0])
            elif 'fit' in r['fit']:
                Y.append(1)
                Y_auc.append([0,1,0])
            else:
                Y.append(2)
                Y_auc.append([0,0,1])

        for r in train_data:
            if (r['item_id'] + '|' + str(r['size'])) in items:
                item_count[r['item_id'] + '|' + str(r['size'])] += 1
                if 'small' in r['fit']:
                    item_small[r['item_id'] + '|' + str(r['size'])] += 1
        for k in item_count:
            item_small[k] = item_small[k]/item_count[k]

        for r in data:
            frac_small.append(item_small[r['item_id'] + '|' + str(r['size'])])

        return np.array(X),np.array(Y), np.array(Y_auc), np.array(item_l), item_n, frac_small
    
    def select_prototype(data):
        chosen_data = []
        X_small,_,_,_,_,_ = prepare_features(data)
        small_mean = np.mean(X_small, axis = 0)
        dist = []
        for i in range(len(data)):
            dist.append((i, sum([(X_small[i][j]-small_mean[j])**2 for j in range(len(small_mean))])))
        if 'fit' in data[0]['fit']:
            small_temp = sorted(dist, key=operator.itemgetter(1))[1300:]
            interval = int(len(data)/5000)
            for i in range(100):
                chosen_data.append(data[small_temp[int(i*interval)][0]])
                interval += 4
        elif 'small' in data[0]['fit']:
            small_temp = sorted(dist, key=operator.itemgetter(1))[1200:]
            interval = int(len(data)/1000)
            for i in range(100):
                chosen_data.append(data[small_temp[int(i*interval)][0]])
                interval += 1.15
        else:
            small_temp = sorted(dist, key=operator.itemgetter(1))[1200:]
            interval = int(len(data)/500)
            for i in range(100):
                chosen_data.append(data[small_temp[int(i*interval)][0]])
                interval += 0.5

        return chosen_data

    small = []; true = []; large= []
    for r in range(len(train_data)):
        if 'small' in train_data[r]['fit']:
            small.append(train_data[r])
        elif 'fit' in train_data[r]['fit']:
            true.append(train_data[r])
        else:
            large.append(train_data[r])

    random.shuffle(small); random.shuffle(true); random.shuffle(large)
    data_training = [];
    data_training.extend(select_prototype(small)); data_training.extend(select_prototype(true)); data_training.extend(select_prototype(large))

    random.shuffle(data_training)
    X_train, Y_train, Y_train_auc, item_l, item_n, frac_small = prepare_features(data_training)
    X_test, Y_test, Y_test_auc, _, _, _ = prepare_features(test_data)

    clf_kLF = LMNN(n_neighbors=53, max_iter=50, n_features_out=X_train.shape[1], verbose=0)
    clf_kLF = clf_kLF.fit(X_train, Y_train)

    from sklearn.metrics import roc_auc_score
    pred = clf_kLF.predict_proba(X_test); AUC = []
    for i in range(3):
        AUC.append(roc_auc_score(Y_test_auc[:,i], pred[:,i], average='weighted'))
    print('Average AUC', np.mean(AUC), AUC)
    return np.mean(AUC)

### Projected Gradient Descent

In [ ]:
def total_loss():
    loss = 0
    for item in item_data:
        for r in item_data[item]:
            s = true_size_cust[user_index[r['user_id']]]
            t = true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]]
            b_i = bias_i[p_item_index[r['item_id']]]
            b_u = bias_u[user_index[r['user_id']]]
            
            if 'small' in r['fit']:
                loss += max(0, 1 - f(alpha,b_i,b_u,s,t) + b_2)
            elif 'fit' in r['fit']:
                loss += max(0, 1 + f(alpha,b_i,b_u,s,t) - b_2)
                loss += max(0, 1 - f(alpha,b_i,b_u,s,t) + b_1)
            elif 'large' in r['fit']:
                loss += max(0, 1 + f(alpha,b_i,b_u,s,t) - b_1)
    return loss

for iterr in range(0,450):
    learning_rate1 = 0.00025
    learning_rate2 = 0.00001/np.sqrt(iterr+1)
    
    grad_b1 = 0
    grad_b2 = 0
    grad_s = np.zeros((len(user_index), K))
    grad_t = np.zeros((len(item_index), K))
    grad_bu = np.zeros(len(user_index))
    grad_bi = np.zeros(len(p_item_index))
    grad_bs = np.zeros(len(sizes))
    grad_w = np.zeros((K+3))
    for r in train_data:
        s = true_size_cust[user_index[r['user_id']]]
        t = true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]]
        b_i = bias_i[p_item_index[r['item_id']]]
        b_u = bias_u[user_index[r['user_id']]]
        
        if 'small' in r['fit']:
            A = 1 - f(alpha,b_i,b_u,s,t) + b_2
            if A>0:
                grad_s[user_index[r['user_id']]] += -1*np.multiply(w[3:],t)
                grad_t[item_index[r['item_id'] + '|' + str(r['size'])]] += -1*np.multiply(w[3:],s)
                
                grad_bu[user_index[r['user_id']]] += -1*w[1]
                grad_bi[p_item_index[r['item_id']]] += -1*w[2]
                grad_w += -1*np.concatenate(([alpha, b_u, b_i], np.multiply(s, t)))
                grad_b2 += 1
        elif 'fit' in r['fit']:
            B = 1 + f(alpha,b_i,b_u,s,t) - b_2
            C = 1 - f(alpha,b_i,b_u,s,t) + b_1
            if B>0:
                grad_s[user_index[r['user_id']]] += np.multiply(w[3:],t)
                grad_t[item_index[r['item_id'] + '|' + str(r['size'])]] += np.multiply(w[3:],s)
                
                grad_bu[user_index[r['user_id']]] += w[1]
                grad_bi[p_item_index[r['item_id']]] += w[2]
                grad_w += np.concatenate(([alpha, b_u, b_i], np.multiply(s, t)))
                grad_b2 += -1
            if C>0:
                grad_s[user_index[r['user_id']]] += -1*np.multiply(w[3:],t)
                grad_t[item_index[r['item_id'] + '|' + str(r['size'])]] += -1*np.multiply(w[3:],s)
                
                grad_bu[user_index[r['user_id']]] += -1*w[1]
                grad_bi[p_item_index[r['item_id']]] += -1*w[2]
                grad_w += -1*np.concatenate(([alpha, b_u, b_i], np.multiply(s, t)))
                grad_b1 += 1
        elif 'large' in r['fit']:
            D = 1 + f(alpha,b_i,b_u,s,t) - b_1
            if D>0:
                grad_s[user_index[r['user_id']]] += np.multiply(w[3:],t)
                grad_t[item_index[r['item_id'] + '|' + str(r['size'])]] += np.multiply(w[3:],s)
                
                grad_bu[user_index[r['user_id']]] += w[1]
                grad_bi[p_item_index[r['item_id']]] += w[2]
                grad_w += np.concatenate(([alpha, b_u, b_i], np.multiply(s, t)))
                grad_b1 += -1

    for i in range(len(user_index)):
        true_size_cust[i] -= learning_rate1*(grad_s[i] + 2*lamda*true_size_cust[i])
        bias_u[i] -= learning_rate1*(grad_bu[i] + 2*lamda*bias_u[i])
        
    for i in range(len(item_index)):
        ## constraint update
        temp = true_size_item[i] - learning_rate1*(grad_t[i] + 2*lamda*true_size_item[i])
        if product_smaller[i] != -1:
            temp = np.maximum(temp, true_size_item[product_smaller[i]])
        if product_larger[i] != -1:
            temp = np.minimum(temp, true_size_item[product_larger[i]])
        true_size_item[i] = temp
        
    for i in range(len(p_item_index)):
        bias_i[i] -= learning_rate1*(grad_bi[i] + 2*lamda*bias_i[i])        
    
    b_1 -= learning_rate2*(grad_b1 + 2*lamda*b_1)
    b_2 -= learning_rate2*(grad_b2 + 2*lamda*b_2)
    w -= learning_rate2*(grad_w + 2*lamda*w)
    if iterr%5 == 0:    
        print(iterr, total_loss(), b_1, b_2,w[:4])
        calc_auc()
        auc = calc_metric_auc()

In [ ]:
calc_metric_auc()